# **1. Portafolio Summary**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime

## **1.1. Descargando cotizacion de precios a cierre**

In [ ]:
# Activos Financieros

assets= ["MSFT", "BA", "AMZN", "GOOG", "SPY"]


fecha_ini = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d').date()
fecha_fin = datetime.datetime.strptime("2025-01-01", '%Y-%m-%d').date()

diferencia_dias = (fecha_fin - fecha_ini).days


# Descargando las cotizaciones

data = yf.download(assets, start=fecha_ini, end=fecha_fin)['Close']
returns = data.pct_change().dropna()


# Risk Free

risk_free = 0.015



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


## **1.2. Retornos diarios y anuales**

In [ ]:
# Calculando los retornos diarios y anuales

daily_return = returns.mean()

Annual_return = returns.mean()*252


# Calculando la volatilidad diaria y anual

daily_sd_return = returns.std()

Annual_sd_return = returns.std()*np.sqrt(252)


## **1.3. Composición del portafolio y Matriz de covarianzas**

In [ ]:
# Pesos del portafolio y posición (capital)

Pesos = [0.25, 0.25, 0.25, 0.25]
Pesos = np.array(Pesos)
Posicion = 1000000


# Matriz de Varianzas y  Covarianzas de los activos financieros

assets_returns = returns.iloc[:,:4]
matriz_var_cov = assets_returns.cov()


## **1.4. Ratio Sharpe**

In [ ]:
# Sharpe Ratio


sharpe_ratios = ((Annual_return - risk_free) / Annual_sd_return)
sharpe_ratios


,0
Ticker,
AMZN,0.605213
BA,-0.019680
GOOG,0.752303
MSFT,0.767056
SPY,0.669794


## **1.5. Ratio Treynor**

In [ ]:
# Calculando el beta de mercado: Asumiendo que el benchmark es SPY.

benchmark_returns = returns['SPY']
betas = {}
for asset in assets:
  asset_returns = returns[asset]
  covariance = np.cov(asset_returns, benchmark_returns)[0, 1]
  benchmark_variance = np.var(benchmark_returns)
  beta = covariance / benchmark_variance
  betas[asset] = beta

print("Betas de los activos en relación al SPY:")

for asset, beta in betas.items():
  print(f"{asset}: {beta:.4f}")


df_betas = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])
df_betas.index.name = 'Ticker'
df_betas = df_betas.sort_index()

Betas de los activos en relación al SPY:
MSFT: 1.1907
BA: 1.4694
AMZN: 1.1048
GOOG: 1.1476
SPY: 1.0008


In [ ]:
# Calculando el ratio de Treynor

treynor_ratios = ((Annual_return - risk_free) / df_betas['Beta'])

treynor_ratios


,0
Ticker,
AMZN,0.197012
BA,-0.006841
GOOG,0.212299
MSFT,0.196473
SPY,0.140527


## **1.6. Ratio de Jensens**

In [ ]:
# Calculando el CAPM para cada activo del portafolio

market_return = returns['SPY'].mean() * 252

capm_returns = {}
for asset, beta in betas.items():
  capm_return = risk_free + beta * (market_return - risk_free)
  capm_returns[asset] = capm_return

df_capm = pd.DataFrame.from_dict(capm_returns, orient='index', columns=['CAPM Return'])
df_capm.index.name = 'Ticker'
df_capm = df_capm.sort_index()
df_capm


,CAPM Return
Ticker,
AMZN,0.170377
BA,0.221648
GOOG,0.176396
MSFT,0.182452
SPY,0.155751


In [ ]:
# Calculando el ratio de Jensens

jensens = Annual_return - df_capm['CAPM Return']

jensens


,0
Ticker,
AMZN,0.062281
BA,-0.216700
GOOG,0.082237
MSFT,0.066479
SPY,-0.000112


# **2. Value at Risk Paramétrico**

In [ ]:
# Varianza, desviación estandar, nivel del confianza, Z Estadístico y tiempo del análisis del portafolio total

Varianza = ((Pesos.dot(matriz_var_cov)).dot(Pesos.T))*100

Desviacion_Estandar = np.sqrt(Varianza)

Nivel_confianza = 0.95

Z_Estadistico = norm.ppf(Nivel_confianza)

Tiempo = 10

# Cálculo del Value at Risk (VaR)

VaR = Posicion * Z_Estadistico * Desviacion_Estandar * np.sqrt(Tiempo)


print('''

    VaR Parámetrico - Datos del Portafolio
    ======================================

    - Varianza            = {:.4f}%

    - Desv. Estandar      = {:.4f}%

    - Nivel de Confianza  = {}%

    - Z-Estadistico       = {:.2f}

    - Tiempo              = {} días

    --------------------------------------

    - Value at Risk       = S/. {:.4f}

'''.format(Varianza, Desviacion_Estandar, Nivel_confianza*100, Z_Estadistico, Tiempo, VaR))



    VaR Parámetrico - Datos del Portafolio

    - Varianza            = 0.0341%

    - Desv. Estandar      = 0.1847%

    - Nivel de Confianza  = 95.0%

    - Z-Estadistico       = 1.64

    - Tiempo              = 10 días

    --------------------------------------

    - Value at Risk       = S/. 960553.6162




# **3. Value at Risk - Histórico**

In [ ]:
assets_returns['Portafolio_Return'] = (assets_returns * Pesos).sum(axis=1)
assets_returns["Portafolio_Return"] = assets_returns["Portafolio_Return"]*Posicion
assets_returns

Ticker,AMZN,BA,GOOG,MSFT,Portafolio_Return
Date,,,,,
2020-01-03,-0.012139,-0.001680,-0.004907,-0.012452,-7794.563779
2020-01-06,0.014886,0.002945,0.024657,0.002585,11268.190966
2020-01-07,0.002092,0.010607,-0.000624,-0.009118,739.204039
2020-01-08,-0.007809,-0.017523,0.007881,0.015928,-380.608216
2020-01-09,0.004799,0.014998,0.011044,0.012493,10833.651988
...,...,...,...,...,...
2024-12-23,0.000622,0.001917,0.015703,-0.003092,3787.551296
2024-12-24,0.017729,0.009286,0.008062,0.009374,11112.500023
2024-12-26,-0.008732,0.005799,-0.002379,-0.002777,-2022.125361


In [ ]:
VaR_Historico = np.percentile(assets_returns["Portafolio_Return"], 5)

print('''

    VaR Parámetrico - Datos del Portafolio
    ======================================

    - Nivel de Confianza  = {}%

    - Z-Estadistico       = {:.2f}

    --------------------------------------

    - Value at Risk - Histórico       = {:.2f}

'''.format(Nivel_confianza*100, Z_Estadistico, VaR_Historico))




    VaR Parámetrico - Datos del Portafolio

    - Nivel de Confianza  = 95.0%

    - Z-Estadistico       = 1.64

    --------------------------------------

    - Value at Risk - Histórico       = -28280.32




# **4. VaR  Monte Carlo**

In [ ]:
# Número de simulaciones
num_simulations = 1000

# Generar retornos simulados para cada activo
simulated_returns = np.random.multivariate_normal(
    mean=assets_returns.mean(),
    cov=assets_returns.cov(),
    size=num_simulations
)

# Calcular el retorno del portafolio para cada simulación
portfolio_returns = np.dot(simulated_returns, Pesos) * Posicion

# Calcular el VaR de Monte Carlo
VaR_MonteCarlo = np.percentile(portfolio_returns, 5)

print('''

    VaR Monte Carlo - Datos del Portafolio
    ======================================

    - Nivel de Confianza  = {}%

    --------------------------------------

    - Value at Risk - Monte Carlo       = {:.2f}

'''.format(Nivel_confianza*100, VaR_MonteCarlo))




    VaR Monte Carlo - Datos del Portafolio

    - Nivel de Confianza  = 95.0%

    --------------------------------------

    - Value at Risk - Monte Carlo       = -29274.66


